### Repeat the import actions from the previous [notebook](feature_analysis.ipynb)

In [1]:
# Command specified for nanohub user, enable lammps executable
# import hublib.use
# %use lammps-03Mar20

# Check the path of executable
from monty.os.path import which
which("lmp_serial")

'/srv/conda/envs/notebook/bin/lmp_serial'

In [2]:
# machine learning interatomic potentials imports
from maml.apps.pes import SNAPotential

# materials properties prediction imports
from pymatgen import Structure, Lattice
from maml.apps.pes import LatticeConstant, ElasticConstant, NudgedElasticBand, DefectFormation

# disable logging information
import logging
logging.disable(logging.CRITICAL)

### Load the paper-implemented ML-IAPs and reproduce the results

Load [Phys. Rev. Mater._(2017)](https://journals.aps.org/prmaterials/abstract/10.1103/PhysRevMaterials.1.043603) Mo snap. Compare the results with that shown in TABLE II.

In [3]:
prm_Mo_snap = SNAPotential.from_config(param_file='./data/paper/PRM_2017/Mo.snapparam', coeff_file='./data/paper/PRM_2017/Mo.snapcoeff')

Reproduce properties prediction (lattice constants, elastic constants, neb barrier, vacancy formation energy)

In [4]:
Mo_cell = Structure.from_spacegroup(sg='Im-3m', species=['Mo'], lattice=Lattice.cubic(3.17), coords=[[0, 0, 0]])

Mo_lc_calculator = LatticeConstant(prm_Mo_snap)
a, b, c = Mo_lc_calculator.calculate([Mo_cell])[0]
print('Mo', 'Lattice a: {:.3f} Å, Lattice b: {:.3f} Å, Lattice c: {:.3f} Å'.format(a, b, c))

Mo Lattice a: 3.160 Å, Lattice b: 3.160 Å, Lattice c: 3.160 Å


In [5]:
Mo_ec_calculator = ElasticConstant(ff_settings=prm_Mo_snap, lattice='bcc', alat=3.17, atom_type='Mo')
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate()
print('Mo', ' C11: {:.0f} GPa'.format(Mo_C11), 'C12: {:.0f} GPa'.format(Mo_C12), 'C44: {:.0f} GPa'.format(Mo_C44))

Mo  C11: 473 GPa C12: 152 GPa C44: 107 GPa


In [6]:
Mo_neb_calcualtor = NudgedElasticBand(ff_settings=prm_Mo_snap, specie='Mo', lattice='bcc', alat=3.16, lmp_exe="lmp_mpi")
barrier = Mo_neb_calcualtor.calculate()
print('Mo NEB barrier: {:.3f} eV'.format(barrier))

Mo NEB barrier: 1.448 eV


In [7]:
Mo_vacancy_calcualtor = DefectFormation(ff_settings=prm_Mo_snap, specie='Mo', lattice='bcc', alat=3.16)
formation_energy = Mo_vacancy_calcualtor.calculate()
print('Mo Vacancy formation energy: {:.2f} eV'.format(formation_energy))

Mo Vacancy formation energy: 2.61 eV


Load [Phys. Rev. B_(2018)](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.98.094104) Ni-Mo snap.

In [8]:
prb_Ni_Mo_snap = SNAPotential.from_config(param_file='./data/paper/PRB_2018/Ni_Mo.snapparam', 
                                          coeff_file='./data/paper/PRB_2018/Ni_Mo.snapcoeff')

In [9]:
Mo_ec_calculator = ElasticConstant(ff_settings=prb_Ni_Mo_snap, lattice='bcc', alat=3.17, atom_type='Mo')
Mo_C11, Mo_C12, Mo_C44, _ = Mo_ec_calculator.calculate()
print('Mo', ' C11: {:.0f} GPa'.format(Mo_C11), 'C12: {:.0f} GPa'.format(Mo_C12), 'C44: {:.0f} GPa'.format(Mo_C44))

Ni_ec_calculator = ElasticConstant(ff_settings=prb_Ni_Mo_snap, lattice='fcc', alat=3.508, atom_type='Ni')
Ni_C11, Ni_C12, Ni_C44, _ = Ni_ec_calculator.calculate()
print('Ni', ' C11: {:.0f} GPa'.format(Ni_C11), 'C12: {:.0f} GPa'.format(Ni_C12), 'C44: {:.0f} GPa'.format(Ni_C44))

Mo  C11: 475 GPa C12: 163 GPa C44: 111 GPa
Ni  C11: 269 GPa C12: 150 GPa C44: 135 GPa
